# Greedy Algorithms Interview Workbook


This notebook curates every greedy-based exercise in the repository. Each section introduces the interview scenario, captures a representative example, outlines why the greedy choice is valid, and embeds the original C++ source so you can revise the implementation side-by-side with the reasoning.


## Greedy Building Blocks


Start with the warm-up problems that reinforce the exchange argument mindset—serving change, matching resources, and keeping decisions locally optimal while maintaining global feasibility.


### Lemonade Change

**Problem statement:** Serve a queue of customers paying with $5, $10, or $20 bills when lemonade costs $5, returning change on the fly or reporting failure if exact change cannot be provided.

**Example:**
```
bills = [5,5,5,10,20]
Output: true
```
**Explanation:** Tracking how many $5 and $10 bills you currently own is sufficient. Always prefer giving change with a $10+$5 combination before three $5 bills so that future customers who hand you $10 have a $5 available.

**Approach highlights:**
* Iterate over each bill, incrementing the count for the bill that just arrived.
* When a $10 appears, ensure there is at least one $5 to return; otherwise the sale fails.
* When a $20 appears, prioritise handing back one $10 and one $5; if impossible, fall back to three $5 bills or return false.

**Complexity:** Time O(n), Space O(1).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution {
public:
    bool lemonadeChange(vector<int>& bills)
    {
        map<int,int> mapping;
        for(auto item : bills)
        {
            mapping[item]++;
            if(item == 10)
            {
                if(mapping[5] > 0)
                {
                    mapping[5]--;
                }
                else
                {
                    return false;
                }
            }
            else if(item == 20)
            {
                if(mapping[10] > 0 && mapping[5] > 0)
                {
                    mapping[10]--;
                    mapping[5]--;
                }
                else if(mapping[5] > 2)
                {
                    mapping[5] -= 3;
                }
                else
                {
                    return false;
                }
            }
        }
        return true;
    }
};


### Assign Cookies

**Problem statement:** Given children with greed factors and cookies with sizes, distribute cookies so that as many children as possible have a cookie at least as large as their greed factor.

**Example:**
```
greed = [1,2,3], cookies = [1,1]
Output: 1
```
**Explanation:** Sorting both lists lets you satisfy the least greedy child first. Each time the current cookie is big enough, assign it and move on; otherwise, discard the small cookie because it cannot help any remaining child.

**Approach highlights:**
* Sort greed factors and cookie sizes in ascending order.
* Walk both arrays with two pointers, advancing the child pointer only when the current cookie suffices.
* Count the number of successful assignments and return it at the end.

**Complexity:** Time O(n log n + m log m), Space O(1) extra aside from sorting.


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int findContentChildren(vector<int>& g,vector<int>& s)
    {
        sort(g.begin(), g.end());
        sort(s.begin(), s.end());
        int i = 0;
        int j = 0;
        int ans = 0;
        while (i < g.size() && j < s.size())
        {
            if (s[j] >= g[i])
            {
                ans++;
                i++;
                j++;
            }
            else
            {
                j++;
            }
        }
        return ans;
    }
};


### Candy Distribution

**Problem statement:** Distribute candies to children standing in a line such that each child has at least one candy and anyone with a higher rating than an adjacent child receives more candies.

**Example:**
```
ratings = [1,0,2]
Output: 5
```
**Explanation:** A left-to-right pass enforces the increasing rule forward, and a right-to-left pass enforces it backward. Taking the maximum candies required by either direction keeps both constraints satisfied with minimal total candies.

**Approach highlights:**
* Initialise everyone with one candy before scanning.
* Sweep from left to right, raising `candies[i]` when `ratings[i] > ratings[i-1]`.
* Sweep from right to left, bumping `candies[i]` to `max(current, right_neighbour + 1)` whenever needed and sum the array.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int candy(vector<int>& ratings)
    {
        int n = ratings.size();
        vector<int> candies(n,1);
        for(int i = 1;i < n;i++)
        {
            if(ratings[i] > ratings[i - 1])
            {
                candies[i] = candies[i - 1] + 1;
            }
        }
        for(int i = n - 2;i >= 0;i--)
        {
            if(ratings[i] > ratings[i + 1])
            {
                candies[i] = max(candies[i],candies[i + 1] + 1);
            }
        }
        return accumulate(candies.begin(),candies.end(),0);
    }
};


### Valid Parenthesis String with Wildcards

**Problem statement:** Check whether a string containing '(', ')' and '*' can be interpreted as a valid parenthesis sequence, where '*' may act as '(' or ')' or be ignored.

**Example:**
```
s = '(*))'
Output: true
```
**Explanation:** Tracking the feasible range of open brackets (`low`, `high`) after each character keeps all possible substitutions compact. If the upper bound ever drops negative we overuse closings, and at the end the lower bound must reach zero.

**Approach highlights:**
* Maintain two counters: the minimum and maximum possible number of unmatched '(' characters so far.
* Increment both counters on '(', decrement both on ')', and adjust `[low, high]` appropriately when a '*' appears.
* Clamp `low` to zero after each step and ensure `high` never becomes negative; success requires `low == 0` at the end.

**Complexity:** Time O(n), Space O(1).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    bool checkValidString(string s)
    {
        int low = 0;
        int high = 0;
        for(auto item : s)
        {
            if(item == '(')
            {
                low++;
                high++;
            }
            else if(item == ')')
            {
                low--;
                high--;
            }
            else if(item == '*')
            {
                low--;
                high++;
            }
            if(high < 0)
            {
                return false;
            }
            low = max(0,low);
        }
        return low == 0;
    }
};


## Interval Scheduling and Timeline Management


These scheduling exercises highlight how sorting by start or finish times unlocks globally optimal schedules for room usage, train platforms, and timeline edits.


### Merge Overlapping Intervals

**Problem statement:** Given a collection of closed intervals, merge all overlapping intervals into the smallest possible set of non-overlapping ranges.

**Example:**
```
intervals = [[1,3],[2,6],[8,10],[15,18]]
Output: [[1,6],[8,10],[15,18]]
```
**Explanation:** Sorting by the start time allows you to extend the current window greedily whenever the next interval starts before the existing end. Otherwise, close out the window and begin a new one.

**Approach highlights:**
* Sort intervals lexicographically by their start.
* Track the active merged interval and compare it with the next range.
* If they overlap, extend the end; if not, store the finished range and reset to the new interval.

**Complexity:** Time O(n log n), Space O(1) extra (ignoring output).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    vector<vector<int>> merge(vector<vector<int>>& intervals)
    {
        sort(intervals.begin(),intervals.end());
        vector<vector<int>> ans;
        int a = intervals[0][0];
        int b = intervals[0][1];
        int n = intervals.size();
        for(int i = 1;i < n;i++)
        {
            int fst = intervals[i][0];
            int lst = intervals[i][1];
            if(a <= fst && fst < b)
            {
                b = max(b,lst);
            }
            else if(fst == b)
            {
                b = lst;
            }
            else
            {
                vector<int> dummy;
                dummy.push_back(a);
                dummy.push_back(b);
                ans.push_back(dummy);
                a = fst;
                b = lst;
            }
        }
        vector<int> dummy;
        dummy.push_back(a);
        dummy.push_back(b);
        ans.push_back(dummy);
        return ans;
    }
};


### Insert and Merge an Interval

**Problem statement:** Insert a new interval into an existing list of non-overlapping, sorted intervals and merge overlaps so the output remains sorted and disjoint.

**Example:**
```
intervals = [[1,3],[6,9]], newInterval = [2,5]
Output: [[1,5],[6,9]]
```
**Explanation:** Because the original list is sorted, you can copy intervals strictly left of the new range, coalesce anything that overlaps the new interval, then append the remaining right-side intervals.

**Approach highlights:**
* Append all intervals that end before the new interval starts.
* Merge the new interval with any overlapping ranges by expanding its start and end bounds.
* Append the merged interval followed by the untouched right-side intervals.

**Complexity:** Time O(n), Space O(1) extra (ignoring output).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    vector<vector<int>> insert(vector<vector<int>>& intervals,vector<int>& newInterval)
    {
        vector<vector<int>> result;
        int n = intervals.size();
        int i = 0;
        // Storing Left Isolated Portion
        while(i < n && intervals[i][1] < newInterval[0])
        {
            result.push_back(intervals[i]);
            i++;
        }
        // Storing Middle Overlapping Portion
        while(i < n && intervals[i][0] <= newInterval[1])
        {
            newInterval[0] = min(newInterval[0],intervals[i][0]);
            newInterval[1] = max(newInterval[1],intervals[i][1]);
            i++;
        }
        result.push_back(newInterval);
        // Storing Right Isolated Portion
        while(i < n)
        {
            result.push_back(intervals[i]);
            i++;
        }
        return result;
    }
};


### Erase Overlapping Intervals

**Problem statement:** Remove the minimum number of intervals so that the remaining set becomes non-overlapping.

**Example:**
```
intervals = [[1,2],[2,3],[3,4],[1,3]]
Output: 1
```
**Explanation:** Sorting by finish time makes it optimal to keep the interval that frees the timeline earliest. Every time a start time conflicts with the previous finish, we count it as a removal.

**Approach highlights:**
* Sort intervals by their end time.
* Scan from earliest finish to latest, counting how many can be kept without overlapping.
* Return total intervals minus the number kept to obtain the removals required.

**Complexity:** Time O(n log n), Space O(1) extra.


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int eraseOverlapIntervals(vector<vector<int>>& intervals)
    {
        int n = intervals.size();
        sort(intervals.begin(), intervals.end(), [](const vector<int>& a, const vector<int>& b) {
            return a[1] < b[1];
        });
        int count = 0;
        int nextFree = -1e9;
        for(auto item : intervals)
        {
            int start = item[0];
            int end = item[1];
            if(start >= nextFree)
            {
                count++;
                nextFree = end;
            }
        }
        return (n - count);
    }
};


### Schedule Maximum Meetings in One Room

**Problem statement:** Given start and end times for meetings, select the maximum number that can fit in a single room without overlap.

**Example:**
```
start = [1,3,0,5,8,5], end = [2,4,6,7,9,9]
Output: 4
```
**Explanation:** Choosing the meeting that finishes first leaves the most room for future meetings. Sorting by end time and greedily picking compatible meetings yields the optimal count.

**Approach highlights:**
* Zip each meeting's start, end, and original index for sorting.
* Sort the tuple list by increasing end time.
* Iterate through the sorted list, accepting a meeting when its start is not earlier than the time the room becomes free.

**Complexity:** Time O(n log n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
private:
    bool compare(tuple<int,int,int>& a,tuple<int,int,int>& b)
    {
        int A = get<1>(a);
        int B = get<1>(b);
        return (A < B);
    }
public:
    int maxMeetings(vector<int>& start,vector<int>& end)
    {
        int n = start.size();
        vector<tuple<int,int,int>> details;
        for(int i = 0;i < n;i++)
        {
            details.push_back({start[i],end[i],(i + 1)});
        }
        sort(details.begin(),details.end(),compare);
        vector<int> meetingOrder;
        int count = 0;
        int nextTime = 0;
        for(auto item : details)
        {
            int s = get<0>(item);
            int e = get<1>(item);
            int p = get<2>(item);
            if(s >= nextTime)
            {
                count++;
                meetingOrder.push_back(p);
                nextTime = e;
            }
        }
        return count;
    }
};


### Minimum Platforms for Trains

**Problem statement:** Given arrival and departure times of trains, compute the minimum number of platforms required so no train waits.

**Example:**
```
arrival = [900, 940, 950, 1100, 1500, 1800]
departure = [910, 1200, 1120, 1130, 1900, 2000]
Output: 3
```
**Explanation:** Sorting arrivals and departures separately lets you simulate time flowing forward. Each arrival increases the platform requirement until a departure frees one.

**Approach highlights:**
* Sort the arrival and departure arrays individually.
* Use two pointers to walk both arrays, incrementing the count on arrivals and decrementing on departures.
* Track the maximum simultaneous trains observed; that is the number of platforms needed.

**Complexity:** Time O(n log n), Space O(1) extra.


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int findPlatform(vector<int>& Arrival,vector<int>& Departure)
    {
        int n = Arrival.size();
        sort(Arrival.begin(),Arrival.end());
        sort(Departure.begin(),Departure.end());
        int i = 0; // For Arrival
        int j = 0; // For Departure
        int c = 0;
        int ans = 0;
        while(i < n)
        {
            if(Arrival[i] <= Departure[j])
            {
                c++;
                i++;
            }
            else
            {
                c--;
                j++;
            }
            ans = max(ans,c);
        }
        return ans;
    }
};


## Reachability and Range Expansion


These array problems rely on stretching pointers as far as possible at each step to decide whether a target can be reached or a container can hold more water.


### Container With Most Water

**Problem statement:** Pick two vertical lines that, together with the x-axis, form a container holding the most water given the heights array.

**Example:**
```
heights = [1,8,6,2,5,4,8,3,7]
Output: 49
```
**Explanation:** The area is limited by the shorter wall, so moving the pointer at the taller wall cannot help. Shifting the shorter wall might find a taller line that increases the area.

**Approach highlights:**
* Initialise two pointers at the ends of the array and compute the area between them.
* Update the maximum area found so far.
* Move the pointer pointing to the shorter line inward, seeking a taller candidate, until the pointers meet.

**Complexity:** Time O(n), Space O(1).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

// array - [1,8,6,2,5,4,8,3,7]

class Solution
{
public:
    int maxArea(vector<int>& height)
    {
        int n = height.size();
        int i = 0;
        int j = n - 1;
        int ans = 0;
        while(i < j)
        {
            int left = height[i];
            int right = height[j];
            ans = max(ans,min(left,right)*(j - i));
            if(left < right)
            {
                i++;
            }
            else
            {
                j--;
            }
        }
        return ans;
    }
};


### Jump Game Reachability

**Problem statement:** Determine whether you can reach the last index of an array when each element denotes the maximum jump length from that position.

**Example:**
```
nums = [2,3,1,1,4]
Output: true
```
**Explanation:** Maintaining the furthest index reachable so far is enough. As long as every index up to that reach can be visited, future positions extend or maintain the reach.

**Approach highlights:**
* Track the furthest index reachable, initialised to 0.
* Iterate through the array while the current index does not exceed the reachable bound.
* Update the reachable bound with `max(reach, i + nums[i])` and return true if it eventually covers the last index.

**Complexity:** Time O(n), Space O(1).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    bool canJump(vector<int>& nums)
    {
        int n = nums.size();
        int reach = 1;
        for(int i = 0;i < n;i++)
        {
            if(reach >= i + 1)
            {
                reach = max(reach,nums[i] + i + 1);
            }
        }
        if(reach < n)
        {
            return false;
        }
        else
        {
            return true;
        }
    }
};


### Jump Game II Minimum Jumps

**Problem statement:** Find the minimum number of jumps needed to reach the last index of the array using jumps of lengths up to `nums[i]`.

**Example:**
```
nums = [2,3,1,1,4]
Output: 2
```
**Explanation:** Scanning the array in layers—where each layer represents all indices reachable with a fixed number of jumps—lets you decide when to increase the jump count while always expanding to the furthest index possible.

**Approach highlights:**
* Maintain the current layer `[l, r]` that can be reached with the existing number of jumps.
* Within the layer, compute the furthest index reachable by another jump.
* Advance to the next layer `l = r + 1, r = farthest` and increment the jump counter until the last index is inside the window.

**Complexity:** Time O(n), Space O(1).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int jump(vector<int>& nums)
    {
        int n = nums.size();
        if(n == 1)
        {
            return 0;
        }
        int minJumps = 0;
        int l = 0;
        int r = 0;
        while(r < n - 1)
        {
            int farthest = 0;
            for(int i = l;i <= r;i++)
            {
                farthest = max(farthest,nums[i] + i);
            }
            l = r + 1;
            r = farthest;
            minJumps++;
        }
        return minJumps;
    }
};


## Job Ordering and Throughput Optimisation


Greedy heuristics also shine in maximising profit and minimising waiting time by sorting jobs on the most important attribute—deadline or processing time.


### Job Sequencing for Maximum Profit

**Problem statement:** Given jobs with deadlines and profits, schedule at most one job per timeslot to maximise total profit.

**Example:**
```
jobs = [[1,2,100],[2,1,19],[3,2,27],[4,1,25],[5,3,15]] (format [id, deadline, profit])
Output: [3,142]
```
**Explanation:** Sorting jobs by profit descending ensures high-profit tasks are considered first. Each job takes the latest available slot before its deadline, using disjoint-set style scanning to find free times.

**Approach highlights:**
* Sort the job list by profit in descending order.
* Track the maximum deadline to size the schedule array of occupied slots.
* For each job, walk backward from its deadline until an empty slot is found; schedule it there and accumulate profit.

**Complexity:** Time O(n log n + n · D) with D bounded by the maximum deadline, Space O(D).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:  
    vector<int> JobScheduling(vector<vector<int>>& Jobs)
    { 
        sort(Jobs.begin(),Jobs.end(), [](const vector<int>& a, const vector<int>& b) {
            return a[2] > b[2];
        });
        int count = 0;
        int maxProfit = 0;
        int maxDeadline = 0;
        for(auto item : Jobs)
        {
            maxDeadline = max(maxDeadline,item[1]);
        }
        vector<int> jobScheduled(maxDeadline + 1,-1);
        for(auto item : Jobs)
        {
            int jobId = item[0];
            int curDeadling = item[1];
            int curProfit = item[2];
            while(jobScheduled[curDeadling] != -1)
            {
                curDeadling--;
            }
            if(curDeadling >= 0)
            {
                jobScheduled[curDeadling] = jobId;
                maxProfit += curProfit;
                count++;
            }
        }
        return {count,maxProfit};
    } 
};


### Shortest Job First Average Waiting Time

**Problem statement:** Given execution times of CPU tasks arriving together, compute the average waiting time assuming you always execute the shortest job available next.

**Example:**
```
executionTime = [4,3,7,1]
Output: 4
```
**Explanation:** Sorting the processing times ensures the shortest jobs run first, which keeps cumulative waiting time minimal—an application of the rearrangement inequality.

**Approach highlights:**
* Sort all job durations in ascending order.
* Accumulate the running total of time spent so far before each job starts.
* Sum the waiting time contributions and divide by the number of jobs for the average.

**Complexity:** Time O(n log n), Space O(1) extra.


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int averageWaitingTime(vector<int>& executionTime)
    {
        int n = executionTime.size();
        sort(executionTime.begin(),executionTime.end());
        int waitingTillNow = 0;
        int totalWaitingTime = 0;
        for(int i = 1;i < n;i++)
        {
            waitingTillNow += executionTime[i - 1];
            totalWaitingTime += waitingTillNow;
        }
        return totalWaitingTime/n;
    }
};
